<a href="https://colab.research.google.com/github/pradeephh3385/Finetuning_LLM/blob/main/Quantization_GGUF_llm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPTQ(generalized post tranining quantization) vs GGML(Generative Graphical Model) vs GGUF(GPT generate unified format) vs AWQ(Activation aware weight quantization)

PTQ(Post Training Quantization)

QAT(Quantization Aware trainig)

LLAMA CPP for QUANTIZATION

In [1]:
! git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 25402, done.
remote: Counting objects: 100% (6796/6796), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 25402 (delta 6619), reused 6516 (delta 6492), pack-reused 18606
Receiving objects: 100% (25402/25402), 46.69 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (18061/18061), done.


In [2]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include -DGGML_CUDA_USE_GRAPHS  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/i

https://huggingface.co/docs/huggingface_hub/en/guides/download

In [3]:
from huggingface_hub import snapshot_download

In [4]:
model_name="Qwen/Qwen1.5-1.8B"

In [5]:
base_model="./original_model"

In [6]:
quantized_path = "./quantized_model"

In [8]:
snapshot_download(repo_id=model_name,local_dir=base_model,local_dir_use_symlinks=False)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/content/original_model'

In [9]:
quantized_model_path=quantized_path+'/FP16.gguf'

In [10]:
quantized_model_path

'./quantized_model/FP16.gguf'

In [11]:
!mkdir quantized_model

In [12]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf


INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 32768
INFO:hf-to-gguf:gguf: embedding length = 2048
INFO:hf-to-gguf:gguf: feed forward length = 5504
INFO:hf-to-gguf:gguf: head count = 16
INFO:hf-to-gguf:gguf: key-value head count = 16
INFO:hf-to-gguf:gguf: rope theta = 1000000.0
INFO:hf-to-gguf:gguf: rms norm epsilon = 1e-06
INFO:hf-to-gguf:gguf: file type = 1
INFO:hf-to-gguf:Set model tokenizer
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:gguf.vocab:Adding 151387 merge(s).
INFO:gguf.vocab:Setting special token type eos to

In [13]:
methods=["q4_k_m"]

In [14]:
quantized_path = "./quantized_model"

In [15]:
import os

for method in methods:
  qtype=f"{quantized_path}/{method.upper()}.gguf"
  print(qtype)

./quantized_model/Q4_K_M.gguf


In [16]:
import os

for method in methods:
  qtype=f"{quantized_path}/{method.upper()}.gguf"
  os.system("./llama.cpp/quantize "+ "./quantized_model"+"/FP16.gguf "+ qtype + " " + method)

In [17]:
! /content/llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt


Log start
main: build = 2993 (902184dd)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1716621971
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 5504
llama_model_loader: - kv   